In [5]:
import numpy as np
from drp_template.math import bound, GPa2Pa, Pa2GPa

Calculate the Voigt-Reuss or Hashin-Shtrikman bounds for 

In [6]:
K_quartz = 36 # GPa
U_quartz = 45 # GPa
f_quartz = 0.8
K_calcite = 75 # GPa
U_calcite = 31 # GPa
f_calcite = 0.2
K_water = 2.2 # GPa
U_water = 1e-20# GPa
f_water = 0.27 # f_water is equal to porosity
porosity = f_water

fractions = np.array([porosity, (1-porosity)*f_quartz, (1-porosity)*f_calcite])
bulk_array = GPa2Pa(np.array([K_water, K_quartz, K_calcite]))
shear_array = GPa2Pa(np.array([U_water, U_quartz, U_calcite]))

# fractions = np.array([f_quartz, f_calcite])
# bulk_array = np.array([K_quartz, K_calcite])
# shear_array = np.array([U_quartz, U_calcite])

print(f"fractions: {fractions}")

fractions: [0.27  0.584 0.146]


Calculate the Voigt (upper) and Reuss (lower) bounds.

In [7]:
k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg = bound(type='voigt-reuss', fractions=fractions, k=bulk_array, u=shear_array)

# convert Pa to GPa
k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg = Pa2GPa(k_voigt, k_reuss, u_voigt, u_reuss, k_avg, u_avg)
print('k_bulk (Voigt: upper; Reuss: lower bounds):', [k_voigt, k_reuss])
print('u (Voigt: upper; Reuss: lower bounds):', [u_voigt, u_reuss])
print('k_avg and u_avg (Hill average for voigt-reuss):', [k_avg, u_avg])

k_bulk (Voigt: upper; Reuss: lower bounds): [32.568000000000005, 7.0974254268492025]
u (Voigt: upper; Reuss: lower bounds): [30.806, 3.703703703703703e-20]
k_avg and u_avg (Hill average for voigt-reuss): [19.832712713424606, 15.403]


Calculate the upper and lower Hashin-Shtrikman bounds.

In [8]:
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = bound(type='hashin-shtrikman', fractions=fractions, k=bulk_array, u=shear_array)

# convert Pa to GPa
k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg = Pa2GPa(k_hs_upper, k_hs_lower, u_hs_upper, u_hs_lower, k_avg, u_avg)
print('k_bulk (upper and lower HS bounds):', [k_hs_upper, k_hs_lower])
print('u (upper and lower HS bounds):', [u_hs_upper, u_hs_lower])
print('k_avg and u_avg (Simple arithmetic avg):', [k_avg, u_avg])

k_bulk (upper and lower HS bounds): [26.913814348904573, 7.0974254268492025]
u (upper and lower HS bounds): [24.61588052450817, 7.759259259259258e-20]
k_avg and u_avg (Simple arithmetic avg): [17.005619887876886, 12.307940262254085]
